### Подготовка датасета по показателю надой молока

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from pylab import rcParams
from IPython.display import display
import math
from prophet import Prophet
pd.set_option('display.max_columns', 130)


import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [2]:
df = pd.read_excel("../../Data cleansing/output data/Просуммированные по категориям с доп регрессорами.xlsx")
df.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

In [3]:
df['Показатель'].unique()

array(['Верблюды', 'КРС', 'Лошади', 'Молоко', 'Овцы и козы', 'Птица',
       'Свиньи', 'Яйца', 'Температура', 'Поголовье: КРС',
       'Поголовье: лошади', 'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки',
       'Цена: Говядина', 'Цена: Баранина', 'Цена: Молоко', 'Цена: Яйца'],
      dtype=object)

In [4]:
df_milk = df[df['Показатель'].isin(['Молоко', 'Температура', 'Осадки', 'Поголовье: КРС', 'Цена: Молоко'])]
df_milk.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
389,Цена: Молоко,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,101.300000,102.211700,102.927182,102.721328,100.358737,97.046899,93.941398,94.317164,94.788749,96.684524,99.004953,101.381072,103.205931,104.031579,105.488021,106.120949,104.210772,101.292870,101.596749,101.596749,102.003136,102.615154,105.898839,106.640131,106.106931,107.380214,107.702354,108.88708,108.233758,107.476122,107.153693,107.046539,107.046539,111.970680,117.233302,117.233302,118.171169,118.407511,117.815473,118.051104,115.808133,113.955203,112.815651,114.733517,114.618784,117.713491,122.775171,122.161295,123.749392,124.120640,124.244761,123.250803,121.648542,120.553706,120.915367,120.189874,120.550444,122.358701,127.253049,128.016567,128.144584,131.604487,130.814860,130.684046,128.201049,128.32925,129.869201,128.830247,129.474398,131.416514,134.307678,135.785062,136.871343,137.966313,138.104280,138.242384,137.274687,138.098335,138.236434,138.236434,140.033507,143.394312,150.277238,151.780011,151.324671,152.989242,155.131092,156.527271,161.223090,166.221005,166.054784,166.386894,168.716310,179.682871,188.127966,195.088700,202.892248,204.312494,215.141056,214.925915,214.925915,210.627397,212.733671,208.266264,205.350536,205.555887,204.939219,205.758976,204.524422,204.728946,199.405994,200.203618,202.005450,202.005450,201.399434,202.003632,204.023669,206.471953,210.188448,216.704290
314,Осадки,АКМОЛИНСКАЯ ОБЛАСТЬ,9.800000,9.800000,8.300000,8.800000,42.800000,16.700000,39.600000,23.500000,14.500000,43.100000,28.100000,15.900000,19.700000,12.700000,19.000000,20.000000,4.900000,88.800000,103.000000,27.900000,46.800000,28.300000,19.700000,40.100000,12.500000,13.400000,18.500000,16.80000,33.700000,50.500000,49.400000,8.700000,8.900000,34.700000,17.700000,8.000000,13.300000,6.000000,17.700000,16.900000,36.600000,56.100000,71.200000,127.800000,20.200000,35.300000,15.600000,8.800000,5.700000,18.300000,14.300000,22.100000,24.900000,37.200000,14.400000,35.000000,74.100000,27.800000,17.700000,18.400000,12.300000,14.200000,3.500000,26.900000,24.000000,9.10000,46.900000,32.200000,16.100000,8.500000,8.300000,9.600000,28.400000,24.700000,21.900000,7.500000,18.800000,13.700000,43.500000,38.200000,17.500000,12.500000,17.300000,4.500000,8.100000,13.000000,9.800000,9.600000,16.800000,68.700000,120.100000,21.500000,3.500000,38.800000,32.100000,11.000000,13.200000,11.400000,22.900000,2.600000,11.800000,18.900000,41.100000,62.400000,71.300000,35.000000,35.700000,32.200000,31.400000,9.900000,11.400000,21.800000,51.900000,61.900000,51.800000,49.000000,15.000000,15.600000,23.700000,16.400000
392,Цена: Молоко,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,100.000000,106.000000,106.000000,106.000000,106.000000

In [5]:
# Step 1: Pivot to wide format (each indicator becomes columns of periods)
df_wide = df_milk.pivot(index="Регион", columns="Показатель")

# Step 2: Flatten multi-level columns: ('2015-01', 'КРС') → 'КРС_2015-01'
df_wide.columns = [f"{col[1]}_{col[0]}" for col in df_wide.columns]
df_wide = df_wide.reset_index()

# Step 3: Melt: one row per region-period-indicator
df_melted = df_wide.melt(id_vars="Регион", var_name="indicator_period", value_name="value")

# Step 4: Extract 'Период' and 'Показатель' from the combined column
df_melted["Период"] = df_melted["indicator_period"].str.extract(r"_(\d{4}-\d{2})$")
df_melted["Показатель"] = df_melted["indicator_period"].str.extract(r"^(.+)_\d{4}-\d{2}")

# Step 5: Pivot again to get final modeling format: one row per region+period, one column per indicator
df_milk = df_melted.pivot_table(index=["Регион", "Период"], columns="Показатель", values="value").reset_index()
print(df_milk.groupby("Регион").size().reset_index(name="Количество строк"))
df_milk

                            Регион  Количество строк
0              АКМОЛИНСКАЯ ОБЛАСТЬ               120
1              АКТЮБИНСКАЯ ОБЛАСТЬ               120
2              АЛМАТИНСКАЯ ОБЛАСТЬ               120
3               АТЫРАУСКАЯ ОБЛАСТЬ               120
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
5                          ГАЛМАТЫ               120
6                          ГАСТАНА               120
7                         ГШЫМКЕНТ                79
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ               120
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ               120
11            КОСТАНАЙСКАЯ ОБЛАСТЬ               120
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ               120
13           МАНГИСТАУСКАЯ ОБЛАСТЬ               120
14                    ОБЛАСТЬ АБАЙ                31
15                  ОБЛАСТЬ ЖЕТІСУ                31
16                  ОБЛАСТЬ ҰЛЫТАУ                31
17            ПАВЛОДАРСКАЯ ОБЛАСТЬ            

Показатель,Регион,Период,Молоко,Осадки,Поголовье: КРС,Температура,Цена: Молоко
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-01,12800.2,9.8,372560.0,-12.490323,100.700000
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-02,15349.3,9.8,399442.0,-10.192857,100.700000
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-03,25086.1,8.3,425605.0,-5.870968,100.297200
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-04,31661.4,8.8,440023.0,4.490000,100.196903
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-05,49129.8,42.8,444647.0,14.574194,96.289224
...,...,...,...,...,...,...,...
2166,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,38641.4,0.0,1120067.0,27.874194,222.272429
2167,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,35950.3,0.5,1101103.0,20.766667,226.717878
2168,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-10,32936.5,13.6,1078583.0,13.200000,229.211775
2169,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-11,30760.6,12.1,1056289.0,6.500000,236.775763


In [6]:
df_milk.sample()

Показатель,Регион,Период,Молоко,Осадки,Поголовье: КРС,Температура,Цена: Молоко
953,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2017-11,20436.2,16.8,341307.0,5.483333,102.807175


In [7]:
df_milk.to_excel("Датасет по молоку Жансая.xlsx", index=False)